# IQPARC Quantum Game Club Q# Practice Set


In [1]:
//Run to import a bunch of utlity functions
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;
open Microsoft.Quantum.Logical; 

# Task 1. Transform: $|0\rangle$ to $|1\rangle$ 
> <font size = 3> Implement operation Transform_1 <br />
Type: (Qubit[]) => Unit <br />
Description: Transforms a qubit array uniformly in the 0 state to qubits in the 1 state <br />


In [1]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;

operation Transform_1(qs : Qubit[]) : Unit {
    for q in qs {
        X(q);
    }
}

operation Execute_Task_1() : Unit {
    use qs = Qubit[3];
    Transform_1(qs);
    AssertMeasurement([PauliZ, PauliZ, PauliZ], qs, One, "Error: Expected Qubits in the 1 state");
    ResetAll(qs);
}


/snippet_.qs(22,10): error QS5022: No identifier with the name "q" exists.
/snippet_.qs(22,16): error QS5022: No identifier with the name "q" exists.
/snippet_.qs(23,12): error QS6301: The type 'h of the given expression is not compatible with the expected return type Result.


In [6]:
%simulate Execute_Task_1

()

# Task 2. Transform: $|0\rangle$ to $|-\rangle$ 
> <font size = 3> Implement operation Transform_2 <br />
Type: (Qubit[]) => Unit <br />
Description: Transforms a qubit array uniformly in the 0 state to qubits in the - state <br />


In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;

operation Transform_2(qs : Qubit[]) : Unit {
    ApplyToEach(X, qs);
    ApplyToEach(H, qs);
}

operation Execute_2() : Unit {
    use qs = Qubit[3];
    Transform(qs);
    AssertMeasurement([PauliX, PauliX, PauliX], qs, One, "Error: Expected Qubits in the - state");

    ResetAll(qs);
}

# Task 3. Write operation to generate N entangled Qubits
> <font size = 3> Implement the operation EPR_N <br />
Type: (Qubit[]) => Result[] <br />
Description: Creates the n qubit bell state on the passed Qubits, then returns the measurements on those qubits </font>

In [ ]:

operation EPR_N(qs : Qubit[]) : Result[] {
    H(qs[0]); // 0 -> +;
    for i in 1 .. Length(qs) - 1 {
        CNOT(qs[0], qs[i]);
    }

    mutable results = new Result[Length(qs)];
    for i in 0 .. Length(qs) - 1 {
        set results w/= i <- M(qs[i]);
        Reset(qs[i]);
    }
    //return ForEach(MResetZ, qs);
    return results;
}

operation Execute_3(N: Int) : Result[] {
    use qs = Qubit[N];
    let res = EPR_N(qs);
    let ctr = new Result[N];
    mutable one = ctr;
    for i in 0 .. Length(ctr)-1 {
        set one w/= i <- One;
    }
    if(EqualA(EqualR, res, ctr) or EqualA(EqualR, res, one)) {
        Message($"Measurement on each qubit yielded identical results");
    } else {
        Message($"Measurement did not yield identical results");
    }
    return res;
}

# Task 4. Write an operation to generate one of the four bell states based off an integer passed to the operation
> <font size = 4> Part A </font> <br />
<font size = 3> Implement operation Bell <br />
Type: (Qubit[], Int) => Unit <br />
Description: Based off the Int passed to Bell put the two Qubits into one of the four Bell states <br />
0 : $|\Phi^+\rangle = |00\rangle + |11\rangle$ <br />
1 : $|\Phi^-\rangle = |00\rangle - |11\rangle$ <br />
2 : $|\Psi^+\rangle = |01\rangle + |10\rangle$ <br />
3 : $|\Psi^-\rangle = |01\rangle - |10\rangle$ <br />

</font>

> <font size = 4> Part B </font> <br />
<font size = 3> Implement operation Execute_4 <br />
Type: (Int) => Unit <br />
Description: Write the entry operation which calls Bell, use Microsoft.Quantum.Diagnositics.DumpMachine to verify the proper bell states are created. <br />
</font>

In [ ]:
open Microsoft.Quantum.Arrays;
open Microsoft.Quantum.Diagnostics;
open Microsoft.Quantum.Canon;
open Microsoft.Quantum.Measurement;

operation Bell(qs: Qubit[], state: Int) : Unit { 
    H(qs[0]);
    CNOT(qs[0], qs[1]);
    if(state == 1) {
        Z(qs[0]);
    } elif (state == 2) {
        X(qs[0]);
    } elif (state == 3) {
        X(qs[0]);
        Z(qs[1]);
    }
}

operation Execute_4(state : Int) : Unit {
    use qs = Qubit[2];
    Bell(qs, state);
    DumpMachine();
    ResetAll(qs);
}

# Task 5. Implement an arbitrary marking oracle and an arbitraty marking to phase converter

> <font size = 4> Part A </font> <br />
<font size = 3> Implement operation Oracle_5 <br />
Type: (Bool[], Qubit[], Qubit) => Unit <br />
Description: Swap the state of the marker qubit if the Qubit array matches the boolean array <br />

</font>

> <font size = 4> Part B </font> <br />
<font size = 3> Implement operation Phase_Converter_5 <br />
Type: (Bool[], Qubit[], (Bool[], Qubit[], Qubit) => Unit) : Unit <br />
Description: Takes the passed marking oracle function and calls it in a way such that it acts as a phase oracle on the Qubit[] array. <br />
Hint: 2 Single Qubit gates are needed to make this work.

</font>

In [ ]:
operation Oracle_5(bitString: Bool[], qs : Qubit[], mark : Qubit) : Unit{ 
    for logic in 0 .. Length(bitString) - 1 {
        if(bitString[logic] == false) {
            X(qs[logic]);
        }
    }
    Controlled X(qs, mark);
    for logic in 0 .. Length(bitString) - 1 {
        if(bitString[logic] == false) {
            X(qs[logic]);
        }
    }
}

//Operation Phase_Converter()
operation Phase_Converter_5(bitString : Bool[], qs: Qubit[], marking_oracle : (Bool[], Qubit[], Qubit) => Unit) : Unit {
    use mark = Qubit();
    X(mark);
    H(mark);
    marking_oracle(bitString, qs, mark);
    Reset(mark);
}

operation Execute_5() : Unit {
    use qs = Qubit[3];
    ApplyToEach(H, qs);
    Phase_Converter([true, false, false], qs, Oracle_6);
    DumpMachine();
    ResetAll(qs);
}

# Task 6. Write operation to create a phase oracle which swaps the phase of the state described in a bitString
> <font size = 3> Implement operation Oracle_6 <br />
Type: (Bool[], Qubit[]) => Unit <br />
Description: Swap the phase of the state which matches the Bool[] array <br />
Hint: A Controlled Z Gate swaps the phase of the $|11..1\rangle$ state



In [ ]:
operation Oracle_6(bitString: Bool[], qs : Qubit[]) : Unit{ 
    for logic in 0 .. Length(bitString) - 1 {
        if(bitString[logic] == false) {
            X(qs[logic]);
        }
    }
    Controlled Z(Most(qs), Tail(qs));
    for logic in 0 .. Length(bitString) - 1 {
        if(bitString[logic] == false) {
            X(qs[logic]);
        }
    }
}

operation Execute_6(cnt : Int) : Result[] {
    use qs = Qubit[cnt];
    ApplyToEach(H, qs);
    let zero = new Bool[cnt];
    Oracle_6(zero, qs);
    DumpMachine();
    return ForEach(MResetZ, qs);
}

In [ ]:
%simulate Execute_6 cnt=5

# Task 7. Implementing Grovers algorithm on $N = 2^5 = 32$ search space with 3 success values

> <font size = 4> Part A </font> <br />
<font size = 3> Implement operation Oracle<br />
Type: (Qubit[]) => Unit <br />
Description: Implement a phase oracle which swaps the phase on the states <br />
$|00010\rangle = |2\rangle$
$|11010\rangle = |26\rangle$ 
$|10100\rangle = |5\rangle$ 

</font>

> <font size = 4> Part B </font> <br />
<font size = 3> Implement operation Diffusion <br />
Type: (Qubit[]) : Unit <br />
Description: Applies Grover's Diffusion operator to the Qubits <br />

</font>

> <font size = 4> Part C </font> <br />
<font size = 3> Implement operation Diffusion <br />
Type: (Int) : Result[] <br />
Description: Prepare Qubits and run Grover's algorithm the number of times specified by the Int<br />

</font>

> <font size = 4> Part D </font> <br />
<font size = 3> When you believe that you have this working contact me for instructions on how to run this on a real quantum computer using microsoft azure. 
</font>

In [ ]:
operation Oracle(qs : Qubit[]) : Unit {
    let p1 = ControlledOnBitString([false, true, false, false, false], Z);
    let p2 = ControlledOnBitString([false, true, false, true, true], Z);
    let p3 = ControlledOnBitString([false, false, true, false, true], Z);
    use ancilla = Qubit();
    X(ancilla);
    p1(qs, ancilla);
    p2(qs, ancilla);
    p3(qs, ancilla);
    X(ancilla);
    Reset(ancilla);
}

operation Diffusion(qs: Qubit[]) : Unit {
    within {
        ApplyToEachA(H, qs);
        ApplyToEachA(X, qs);
    } apply {
        Controlled Z (Most(qs), Tail(qs));
    }
}

operation RunGroverSearch(iterations : Int) : Result[]{
    use qs = Qubit[5];
    ApplyToEach(H, qs);
    for _ in 1 .. iterations {
        DumpMachine();
        Oracle(qs);
        Diffusion(qs);
    }
    DumpMachine();
    return ForEach(MResetZ, qs);
}

# Task 8. Implement a quantum key exchange system between sender and receiver
> <font size = 4> Part A </font> <br />
<font size = 3> Implement operation Key_Exchange<br />
Type: (Int) => Result[] <br />
Description: Think of this operation as the communication mediator for our Sender and Receiver. It will be in charge of preparing qubits, passing them between the Sender and Receiver, and keeping track of the measurement basis + sent state which are returned from the Receiver and Sender operations. It then uses that information to decode the securely exchanged key. 

</font>

> <font size = 4> Part B </font> <br />
<font size = 3> Implement operation Receiver <br />
Type: (Qubit) => (Result, Result) <br />
Description: Given two qubits chooses randomly between 2 states and basis' in order to prepare for QKE. Must keep track of state and basis <br />

</font>

> <font size = 4> Part C </font> <br />
<font size = 3> Implement operation Sender <br />
Type: (Qubit) : (Result, Result) <br />
Description: Takes to qubits and performs a measurement in 2 basis randomly and returns the basis and result<br />
</font>

In [ ]:
operation Key_Exchange(n_bits : Int) : Result[] {
    mutable sent = new Result[n_bits];
    mutable received = new Result[n_bits];
    mutable basis_s = new Result[n_bits]; //0 = Z axis measurement, 1 = X axis measurement
    mutable basis_r = new Result[n_bits];
    for i in 0 .. n_bits - 1 {
        use q = Qubit() {
            let (basis_value_s, sent_value) = Sender(q);
            set basis_s w/= i <- basis_value_s;
            set sent w/= i <- sent_value;
            let (basis_value_r, received_value) = Receiver(q);
            set basis_r w/= i <- basis_value_r;
            set received w/= i <- received_value;
            Reset(q);
        }
    }
    mutable key = new Result[1];
    for i in 0 .. n_bits -1 {
        if(basis_s[i] == basis_r[i]) {
            if(sent[i] == received[i]) {
                set key = key + [sent[i]];
            }
        }
    }
    Message($"Sent: {sent}, basis_s: {basis_s}");
    Message($"Received: {received}, basis_r: {basis_r}");
    return key;
}

operation Receiver(q : Qubit) : (Result, Result) {
    use rand = Qubit() {
        H(rand);
        let basis = M(rand);
        if(basis == One) {
            H(q);

        }
        let received = M(q);
        Reset(rand);
        return (basis, received);
    }
}
operation Sender(q : Qubit) : (Result, Result) {
    use rand = Qubit() {
        //Randomly select Basis and Bit state
        H(rand);
        let basis = M(rand);
        H(rand);
        let sent = M(rand);

        if(basis == One) {
            H(q);
            if(sent == One) {
                Z(q);
            }
        } else {
            if(sent == One) {
                X(q);
            }
        }
        Reset(rand);
        return (basis, sent);
    }
}

# Other potential project ideas
<font size=3>

* Implement the quantum fourier transform on a qubit array=
* Use the QFT to implement shors algorithm
* Implement the Quantum Teleportation protocol
 <font>